In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e5/sample_submission.csv
/kaggle/input/playground-series-s5e5/train.csv
/kaggle/input/playground-series-s5e5/test.csv


In [2]:
train=pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv')

In [3]:
print(train.shape)
print(test.shape)

(750000, 9)
(250000, 8)


In [4]:
train.columns

Index(['id', 'Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate',
       'Body_Temp', 'Calories'],
      dtype='object')

In [5]:
print(train.isnull().sum())
print(train.describe())
print(train.info())

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64
                  id            Age         Height         Weight  \
count  750000.000000  750000.000000  750000.000000  750000.000000   
mean   374999.500000      41.420404     174.697685      75.145668   
std    216506.495284      15.175049      12.824496      13.982704   
min         0.000000      20.000000     126.000000      36.000000   
25%    187499.750000      28.000000     164.000000      63.000000   
50%    374999.500000      40.000000     174.000000      74.000000   
75%    562499.250000      52.000000     185.000000      87.000000   
max    749999.000000      79.000000     222.000000     132.000000   

            Duration     Heart_Rate      Body_Temp       Calories  
count  750000.000000  750000.000000  750000.000000  750000.000000  
mean       15.421015      95.483995      40.036253      88.282781  
std         8.354

In [6]:
print(test.isnull().sum())
print(test.describe())
print(test.info())

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
dtype: int64
                  id            Age         Height         Weight  \
count  250000.000000  250000.000000  250000.000000  250000.000000   
mean   874999.500000      41.452464     174.725624      75.147712   
std     72168.927986      15.177769      12.822039      13.979513   
min    750000.000000      20.000000     127.000000      39.000000   
25%    812499.750000      28.000000     164.000000      63.000000   
50%    874999.500000      40.000000     174.000000      74.000000   
75%    937499.250000      52.000000     185.000000      87.000000   
max    999999.000000      79.000000     219.000000     126.000000   

            Duration     Heart_Rate      Body_Temp  
count  250000.000000  250000.000000  250000.000000  
mean       15.415428      95.479084      40.036093  
std         8.349133       9.450161       0.778448  
min         1.000000     

In [7]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
le=LabelEncoder()
sc=StandardScaler()

In [8]:
train['Sex']=le.fit_transform(train['Sex'])
test['Sex']=le.transform(test['Sex'])

In [9]:
features = ['Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
X=train[features]
y=train['Calories']
X_test=test[features]
X_scaled=sc.fit_transform(train[features])
X_test_scaled=sc.transform(test[features])

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,KFold
from sklearn.metrics import mean_squared_log_error

In [11]:
params = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2'],
    'n_jobs': [-1]}

In [12]:
search=RandomizedSearchCV(RandomForestRegressor(), param_distributions=params,  
                         random_state=42 , n_jobs=-1)

In [13]:
search.fit(X.iloc[:20000],y.iloc[:20000].values.ravel())
best_params = search.best_params_
print(best_params)

{'n_jobs': -1, 'n_estimators': 150, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None}


In [14]:
FOLDS = 5
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)

oof_preds_rf = np.zeros(len(train))
test_preds_rf = np.zeros(len(test))

rf_params = {**best_params}


In [15]:
for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
    print(f"### Fold {fold+1} is processing...###")

    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx].values.ravel()
    X_valid, y_valid = X.iloc[valid_idx], y.iloc[valid_idx].values.ravel()
    X_test = test[features]

    model=RandomForestRegressor(**rf_params)
    model.fit(X_train,y_train)
    oof_preds_rf[valid_idx] = model.predict(X_valid)
    test_preds_rf += model.predict(X_test) / FOLDS
    print(f"Fold {fold+1} is processed!\n")
    
rmsle = np.sqrt(mean_squared_log_error(y, oof_preds_rf))

### Fold 1 is processing...###
Fold 1 is processed!

### Fold 2 is processing...###
Fold 2 is processed!

### Fold 3 is processing...###
Fold 3 is processed!

### Fold 4 is processing...###
Fold 4 is processed!

### Fold 5 is processing...###
Fold 5 is processed!



In [16]:
rmsle

0.06326410100573124

In [17]:
sub = pd.read_csv('/kaggle/input/playground-series-s5e5/sample_submission.csv')
sub["Calories"] = test_preds_rf
sub.to_csv("submission.csv", index=False)